In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install h2o
!pip install sklearn

In [ ]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/data/voted/train.csv', index_col=0)
test_data = pd.read_csv('/content/drive/MyDrive/data/voted/test_x.csv', index_col=0)
submission = pd.read_csv('/content/drive/MyDrive/data/voted/sample_submission.csv', index_col=0)

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,21 mins 35 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.7
H2O_cluster_version_age:,9 days
H2O_cluster_name:,H2O_from_python_unknownUser_8w60rc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.342 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [ ]:
# 레이블 설정
label = train_data['voted']
label.describe()

# 레이블을 제외한 나머지 데이터를 Feautre로 설정
feature = train_data.drop('voted', axis=1)

In [ ]:
y = "voted"
x = list(train_data.columns)
x.remove(y)

In [ ]:
# 데이터 스플릿
train, valid = train_test_split(train_data,
                                test_size=0.2, 
                                shuffle=True)
h2o_train = h2o.H2OFrame(train)
h2o_valid = h2o.H2OFrame(valid)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
h2o_train[y] = h2o_train[y].asfactor()
h2o_valid[y] = h2o_valid[y].asfactor()

In [ ]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x = x, y = y, training_frame=h2o_train, leaderboard_frame=h2o_valid)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
leaderboard = aml.leaderboard
performance = aml.leader.model_performance(h2o_valid)

In [ ]:
print(leaderboard)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20210912_052254,0.776942,0.550707,0.83085,0.331984,0.43466,0.188929
StackedEnsemble_BestOfFamily_AutoML_20210912_052254,0.775313,0.552543,0.829404,0.326554,0.435476,0.189639
GBM_1_AutoML_20210912_052254,0.774132,0.553622,0.828264,0.33888,0.435936,0.190041
GBM_2_AutoML_20210912_052254,0.773295,0.554234,0.828355,0.343002,0.436335,0.190388
GBM_3_AutoML_20210912_052254,0.773184,0.554237,0.828389,0.352981,0.436357,0.190407
GBM_grid__1_AutoML_20210912_052254_model_1,0.772749,0.555117,0.828117,0.336085,0.436405,0.19045
XGBoost_3_AutoML_20210912_052254,0.768189,0.558743,0.823037,0.353431,0.438521,0.192301
GBM_5_AutoML_20210912_052254,0.768041,0.557961,0.82416,0.342638,0.438126,0.191954
GBM_4_AutoML_20210912_052254,0.767746,0.557337,0.825999,0.354952,0.438104,0.191935
XGBoost_grid__1_AutoML_20210912_052254_model_4,0.767703,0.557963,0.823012,0.35952,0.438271,0.192082


In [ ]:
model_id   =aml.leader.model_id  # 최고 모델 명
accuracy   =performance.accuracy()  # 정확도
precision  =performance.precision()  # precision
recall     =performance.recall()  # recall
F1         =performance.F1()  # f1
auc        =performance.auc()  # auc
variable_importance=aml.leader.varimp()  # 중요한 입력 변수

print(model_id, accuracy, precision, recall, F1, auc, variable_importance)
print(performance)

StackedEnsemble_AllModels_AutoML_20210912_052254 [[0.49337727647401947, 0.705391457120896]] [[0.989445749046625, 1.0]] [[0.13829759756339416, 1.0]] [[0.38894845914928555, 0.7383836557584523]] 0.7769416967716806 None

ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.18892934862703145
RMSE: 0.4346600379917982
LogLoss: 0.5507074719265396
Null degrees of freedom: 9106
Residual degrees of freedom: 9090
Null deviance: 12545.722257650632
Residual deviance: 10030.585893669993
AIC: 10064.585893669993
AUC: 0.7769416967716806
AUCPR: 0.8308498044575198
Gini: 0.5538833935433611

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38894845914928555: 


,,1,2,Error,Rate
0,1,2129.0,2000.0,0.4844,(2000.0/4129.0)
1,2,894.0,4084.0,0.1796,(894.0/4978.0)
2,Total,3023.0,6084.0,0.3178,(2894.0/9107.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.388948,0.738384,276.0
1,max f2,0.138298,0.857743,398.0
2,max f0point5,0.552550,0.754357,184.0
3,max accuracy,0.493377,0.705391,215.0
4,max precision,0.989446,1.000000,0.0
5,max recall,0.138298,1.000000,398.0
6,max specificity,0.989446,1.000000,0.0
7,max absolute_mcc,0.744411,0.433221,108.0
8,max min_per_class_accuracy,0.462308,0.703696,232.0
9,max mean_per_class_accuracy,0.530496,0.712564,195.0



Gains/Lift Table: Avg response rate: 54.66 %, avg score: 54.77 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010102,0.981632,1.829450,1.829450,1.000000,0.983297,1.000000,0.983297,0.018481,0.018481,82.944958,82.944958,0.018481
1,2,0.020094,0.980205,1.789242,1.809456,0.978022,0.980951,0.989071,0.982130,0.017879,0.036360,78.924190,80.945559,0.035876
2,3,0.030087,0.978736,1.829450,1.816096,1.000000,0.979428,0.992701,0.981233,0.018280,0.054640,82.944958,81.609593,0.054156
3,4,0.040079,0.977443,1.789242,1.809401,0.978022,0.978029,0.989041,0.980434,0.017879,0.072519,78.924190,80.940082,0.071550
4,5,0.050071,0.975876,1.809346,1.809390,0.989011,0.976678,0.989035,0.979684,0.018080,0.090599,80.934574,80.938982,0.089388
5,6,0.100033,0.948388,1.805325,1.807360,0.986813,0.970358,0.987925,0.975026,0.090197,0.180796,80.532497,80.735963,0.178131
6,7,0.149995,0.870865,1.624390,1.746415,0.887912,0.901956,0.954612,0.950687,0.081157,0.261953,62.439039,74.641453,0.246937
7,8,0.200066,0.821913,1.588733,1.706951,0.868421,0.846379,0.933041,0.924582,0.079550,0.341503,58.873253,70.695076,0.311956
8,9,0.299989,0.699196,1.379124,1.597755,0.753846,0.766570,0.873353,0.871950,0.137806,0.479309,37.912353,59.775501,0.395511
9,10,0.400022,0.547537,1.122571,1.478926,0.613611,0.613025,0.808400,0.807201,0.112294,0.591603,12.257115,47.892644,0.422555


In [ ]:
test_data = h2o.H2OFrame(test_data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
predictions = aml.predict(test_data)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [ ]:
predictions = predictions.as_data_frame()

In [ ]:
predictions.head()

,predict,p1,p2
0,2,0.330223,0.669777
1,2,0.088118,0.911882
2,2,0.450934,0.549066
3,1,0.829009,0.170991
4,2,0.271225,0.728775


In [ ]:
submission['voted'] = predictions['predict']

In [ ]:
submission.to_csv('/content/drive/MyDrive/data/voted/submission2.csv')